# Fitting Description

## Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/digital_breakthrough/task_3

/content/drive/MyDrive/digital_breakthrough/task_3


In [4]:
!cat requirements.txt
!pip install -r requirements.txt

albumentations==0.5.2
tensorflow==2.5.0
numpy>=1.19.5
torch>=1.7.1
pandas>=1.2.4
torchvision>=0.8.2
opencv-python
PyYAML
tqdm==4.56.0
scikit-image
scikit-learn
scipy
matplotlib
python-json-logger>=0.1.11
jupyterlab
seaborn
grad-cam
ttach
transformers
omegaconfCollecting albumentations==0.5.2
     |████████████████████████████████| 72 kB 778 kB/s 
     |████████████████████████████████| 11.5 MB 23.8 MB/s 
     |████████████████████████████████| 72 kB 415 kB/s 
     |████████████████████████████████| 8.6 MB 54.9 MB/s 
     |████████████████████████████████| 1.7 MB 71.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.6 MB 56.0 MB/s 
     |████████████████████████████████| 74 kB 3.8 MB/s 
     |████████████████████████████████| 37.1 MB 81 kB/s 
     |████████████████████████████████| 948 kB 63.8 MB/s 
     |████████████████████████████████| 392 kB 81.3 MB/s 
     |█

In [1]:
cd /content/drive/MyDrive/digital_breakthrough/task_3

/content/drive/MyDrive/digital_breakthrough/task_3


## Load Data

In [2]:
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
sys.path.append('.')
from definitions import ROOT_DIR
from airotica.utils import read_image

In [3]:
DATA_PATH = ROOT_DIR / 'data'
TRAIN_IMAGES = DATA_PATH / 'images'
DOWNLOADED_TRAIN_IMAGES = DATA_PATH / 'downloaded_images'

In [4]:
train = pd.read_csv(DATA_PATH / 'train.csv')
train_url_only = pd.read_csv(DATA_PATH / 'train_url_only.csv')
train_url_loaded_images = pd.read_csv(DATA_PATH / 'train_loaded_images.csv')
test = pd.read_csv(DATA_PATH / 'test.csv')
sample_submission = pd.read_csv(DATA_PATH / 'sample_submission.csv')

In [5]:
import os
from os import listdir
train_images = listdir(TRAIN_IMAGES)
guid_train_images = [f.split('.')[0] for f in train_images if os.path.getsize(f"{str(TRAIN_IMAGES / f)}") != 0]

In [6]:
dummy = test[test.guid.isin(guid_train_images)]
test_only_description = test[~(test.guid.isin(guid_train_images))]
test_only_images = dummy[dummy.description.isna()]
test_images_and_description = dummy[~(dummy.description.isna())]
print('without image or description:', len(test_only_description[test_only_description.description.isna()]))
print('only description:', test_only_description.shape[0])
print('only images:', test_only_images.shape[0])
print('images and description:', test_images_and_description.shape[0])

without image or description: 0
only description: 106
only images: 547
images and description: 570


 ## FitImages

### Prepared Data

In [7]:
train_url_loaded_images['typology'] = train_url_loaded_images.typology.replace(
    {'предметы прикладного искусства, быта и этнографии ': 'предметы прикладного искусства, быта и этнографии'}
    )

In [8]:
train_labels = train.typology.unique()
typology_to_label = dict(zip(sorted(train_labels), range(len(train_labels))))

In [9]:
train_labels_url = train_url_loaded_images[~(train_url_loaded_images.typology.isna())].typology.unique()
typology_to_label_url = dict(zip(sorted(train_labels_url), range(len(train_labels_url))))

In [10]:
train_url_only_train_labels = train_url_loaded_images[train_url_loaded_images.typology.isin(typology_to_label.keys())]
print(len(train_url_only_train_labels))

175076


In [11]:
dummy_train = train[train.guid.isin(guid_train_images)]
dummy_train.loc[:, 'add'] = 0
train_url_only_train_labels.loc[:, 'add'] = 1
full_train = pd.concat((dummy_train, train_url_only_train_labels.drop('url', axis=1)), axis=0)
full_train = full_train[~(full_train.typology.isna())]
full_train['label'] = full_train['typology'].map(typology_to_label)
full_train

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,guid,description,typology,add,label
0,c84c547b-c5c5-45cf-9199-736df1301124,Монета. Екатерина II. Две копейки. 1789 г.,предметы нумизматики,0,7
1,af6fb03f-3d31-484f-ba9d-51e7b4ef55b9,Владельческий конволют. Собрание сочинений. / ...,редкие книги,0,12
2,5ad52d30-8239-4b41-bd56-da99ab5a555b,"Медаль ВДНХ ""За успехи в народном хозяйстве СС...",предметы нумизматики,0,7
3,6ce2ecbe-80d2-45d1-8a4c-7599950a7792,Монета. Михаил Федорович. Копейка,предметы нумизматики,0,7
4,73a48c2d-12c3-40da-b071-f9abd5bed64b,Временное удостоверение №12849 Охотина Р.А. о ...,документы,0,1
...,...,...,...,...,...
175119,0313dde2-b5bd-4a1a-b7ef-63436ad975ad,Фото ч/б матов. групповое с фигурными краями ...,фотографии и негативы,1,14
175120,669ecd91-4197-4734-a9a9-7cb79d4af9d3,Открытое письмо «Петродворец. Вид на Аллею фо...,предметы печатной продукции,1,8
175121,07ca9b7c-efea-49e3-8db8-e69e9a079bb4,Фото черно-белое. Художник Алексей Як. Кольцов...,фотографии и негативы,1,14
175122,c0df1cb4-1ceb-47ea-873e-6d96fc5a1a95,Грампластинка.,прочие,1,11


In [ ]:
full_train.to_csv('./data/full_images_data.csv', index=False)

### Model

In [12]:
import os
import sys
import torch
import yaml
import logging
import numpy as np
import random
from tqdm import tqdm
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split

sys.path.append('.')
from definitions import ROOT_DIR
from airotica.utils import save_checkpoint, convert_dict_to_tuple
from airotica.models import load_model
from airotica.loss_function import get_loss
from airotica.optimizers import (
    get_optimizer,
    get_scheduler
)
from airotica.dataset import get_data_loaders
from airotica.train import train as train_model, validation

%matplotlib inline
%load_ext autoreload
%autoreload 2

log = logging.getLogger(__name__)

{"asctime": "2021-08-01 12:11:01", "name": "matplotlib.pyplot", "filename": "pyplot.py", "levelname": "DEBUG", "message": "Loaded backend module://ipykernel.pylab.backend_inline version unknown."}


In [13]:
DATA_PATH = ROOT_DIR / 'data'
CONFIG_PATH = ROOT_DIR / 'config5.yml'

In [14]:
with open(CONFIG_PATH) as f:
    data = yaml.safe_load(f)
config = convert_dict_to_tuple(dictionary=data)

device_name = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_name)
print(f'device: {device_name}')

device: cuda


In [15]:
seed = config.dataset.seed
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

os.environ['CUDA_VISIBLE_DEVICES'] = config.cuda_id

In [16]:
print("Loading model...")
net = load_model(config, 
                 device=device_name)
print("Done.")
criterion, criterion_val = get_loss(config, device=device_name)
optimizer = get_optimizer(config, 'SGD', net)

n_epoch = 20
scheduler = get_scheduler(config, optimizer)
train_epoch = tqdm(range(config.train.n_epoch),
                   dynamic_ncols=True,
                   desc='Epochs',
                   position=0)

Loading model...
{"asctime": "2021-08-01 12:11:01", "name": "airotica.models", "filename": "models.py", "levelname": "INFO", "message": "ResNext101_32x8d"}


Downloading: "https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth" to /root/.cache/torch/hub/checkpoints/resnext101_32x8d-8ba56ff5.pth


  0%|          | 0.00/340M [00:00<?, ?B/s]

Done.
{"asctime": "2021-08-01 12:11:11", "name": "airotica.optimizers", "filename": "optimizers.py", "levelname": "INFO", "message": "0.002"}
{"asctime": "2021-08-01 12:11:11", "name": "airotica.optimizers", "filename": "optimizers.py", "levelname": "INFO", "message": "Opt: SGD"}


Epochs:   0%|          | 0/21 [00:00<?, ?it/s]

### Data

In [17]:
trained = pd.read_csv('./data/full_images_data.csv')

In [18]:
label_to_typology = {v: k for k, v in typology_to_label.items()}

In [19]:
for i in range(15):
    globals()[f'label_{i}'] = trained[trained['label'] == i]
    print(globals()[f'label_{i}'].shape[0], label_to_typology[i], i)

14027 графика 0
24186 документы 1
3365 живопись 2
695 оружие 3
22153 предметы археологии 4
4074 предметы естественнонаучной коллекции 5
1431 предметы минералогической коллекции 6
21596 предметы нумизматики 7
19836 предметы печатной продукции 8
21262 предметы прикладного искусства, быта и этнографии 9
1532 предметы техники 10
8244 прочие 11
7746 редкие книги 12
505 скульптура 13
28772 фотографии и негативы 14


In [24]:
n = 1000
full = pd.concat(
    [
     label_3,
     label_4[:5000],
     label_6,
     label_10,
     label_13,
     label_0[:n],
     label_1[:n],
     label_2[:n],
     label_5[:n],
     label_7[:n],
     label_8[:n],
     label_9[:n],
     label_11[:n],
     label_12[:n],
     label_14[:n]
    ]
)

In [27]:
mapper = {3: 0,
          4: 1,
          6: 2,
          10: 3,
          13: 4,
          0: 5,
          1: 5,
          2: 5,
          5: 5,
          7: 5,
          8: 5,
          9: 5,
          11: 5,
          12: 5,
          14: 5,
          }

In [28]:
full['label'] = full['label'].map(mapper)

In [38]:
failed_guid = ['5c989b07-c6a2-4c7b-bd48-e6f9bca7eb27', 
               'cbd73791-e011-40e1-8cc7-f72a745bcebb', 
               'bebd7545-b84c-4af1-84bd-425e2b6313a9',
               'baa188a2-d630-489f-b92e-6cfaba762b0b',
               '4c39c840-cf9f-4363-82bf-74ea084c4f0d',
               '94c866c7-6230-409f-a5f9-b50f976ab62f',
               '46ecbdac-521d-4967-8d04-78e1811d3403',
               '60cacca9-e773-4932-83ca-e4d61bf5c134',
               '76975b28-9dfd-4cb6-bb54-c10a71b886a8',
               'b8165817-1662-4c17-ab53-fd0525537140',
               '99d35800-949b-445f-a018-d2e60b67d9c6',
               '852f75be-19bf-4148-932c-1637facbe8bc',
               'b8165817-1662-4c17-ab53-fd0525537140',
               'a733aa01-0e72-4a69-9045-6d262110c766',
               '78d9b613-9ced-4c65-9cb0-6b2d4dc8006c',
               '26e283c4-5919-4586-912c-3a45bfe290da',
               '1e7d2f58-d625-415c-ab3e-68b13ac46649']

full_train_2 = full[~(full.guid.isin(failed_guid))].reset_index(drop=True)

In [39]:
X_train, X_valid = train_test_split(full_train_2, test_size=0.15, random_state=42)
print(f'train: {X_train.shape[0]}, valid: {X_valid.shape[0]}')

train: 16280, valid: 2873


In [40]:
dt, dv = get_data_loaders(train_data=X_train, 
                          valid_data=X_valid, 
                          config=config, 
                          img_path=TRAIN_IMAGES, 
                          download_img_path=DOWNLOADED_TRAIN_IMAGES, 
                          use_data=True)

{"asctime": "2021-08-01 12:48:16", "name": "airotica.dataset", "filename": "dataset.py", "levelname": "INFO", "message": "Preparing train reader..."}
{"asctime": "2021-08-01 12:48:16", "name": "airotica.dataset", "filename": "dataset.py", "levelname": "INFO", "message": "Done."}
{"asctime": "2021-08-01 12:48:16", "name": "airotica.dataset", "filename": "dataset.py", "levelname": "INFO", "message": "here"}
{"asctime": "2021-08-01 12:48:16", "name": "airotica.dataset", "filename": "dataset.py", "levelname": "INFO", "message": "Preparing valid reader..."}
{"asctime": "2021-08-01 12:48:16", "name": "airotica.dataset", "filename": "dataset.py", "levelname": "INFO", "message": "Done."}


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [41]:
out_dir = str(ROOT_DIR / os.path.join(config.outdir, 'classes_3.4.6.10.13'))
print("Savedir: {}".format(out_dir))
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

Savedir: /content/drive/MyDrive/digital_breakthrough/task_3/EXPERIMENTS/classes_3.4.6.10.13


In [42]:
f1_best = 0
for epoch in train_epoch:
    train_model(net, dt, criterion, optimizer, config, epoch)
    f1_val = validation(net, dv, criterion_val, epoch)
    if f1_val > f1_best:
        save_checkpoint(net, optimizer, scheduler, epoch, out_dir, f1_val)
        f1_best = f1_val
    scheduler.step()


Train:   0%|          | 0/339 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))

Train: 100%|██████████| 339/339 [07:50<00:00,  1.39s/it]


Val:   0%|          | 0/60 [00:00<?, ?it/s]


    Train process of epoch: 0 is done;
    loss: 0.4646; acc: 0.86, f1: 0.85393165
    




Val:   2%|▏         | 1/60 [00:28<28:16, 28.75s/it]

Val:   3%|▎         | 2/60 [00:29<11:59, 12.41s/it]

Val:   5%|▌         | 3/60 [00:30<06:32,  6.88s/it]

Val:   7%|▋         | 4/60 [00:30<03:59,  4.28s/it]

Val:   8%|▊         | 5/60 [00:30<02:36,  2.85s/it]

Val:  10%|█         | 6/60 [00:30<01:47,  1.98s/it]

Val:  12%|█▏        | 7/60 [00:31<01:15,  1.43s/it]

Val:  13%|█▎        | 8/60 [00:31<00:55,  1.07s/it]

Val:  15%|█▌        | 9/60 [00:31<00:42,  1.20it/s]

Val:  17%|█▋        | 10/60 [00:32<00:33,  1.49it/s]

Val:  18%|█▊        | 11/60 [00:32<00:27,  1.79it/s]

Val:  20%|██        | 12/60 [00:32<00:23,  2.08it/s]

Val:  22%|██▏       | 13/60 [00:33<00:19,  2.35it/s]

Val:  23%|██▎       | 14/60 [00:33<00:17,  2.60it/s]

Val:  25%|██▌       | 15/60 [00:33<00:16,  2.79it/s]

Val:  27%|██▋       | 16/60 [00:33<00:14,  2.94it/s]

Val:  28%|██▊       | 17/60 [00:54<04:38,  6.47s/it]

Val:  30%|███       | 18/60 [00:54<03:13,  4.61s/it]

Val:  32%|███▏      | 19/60 [00:55<


        Validation of epoch: 0 is done; 
        loss: 0.3689; acc: 0.88, f1: 0.86527716
        



Train: 100%|██████████| 339/339 [06:00<00:00,  1.06s/it]


Val:   0%|          | 0/60 [00:00<?, ?it/s]

...


    Train process of epoch: 6 is done;
    loss: 0.1031; acc: 0.99, f1: 0.99099954
    




Val:   2%|▏         | 1/60 [00:10<10:41, 10.87s/it]

Val:   3%|▎         | 2/60 [00:18<08:52,  9.18s/it]

Val:   5%|▌         | 3/60 [00:19<04:52,  5.14s/it]

Val:   7%|▋         | 4/60 [00:19<03:00,  3.23s/it]

Val:   8%|▊         | 5/60 [00:19<01:59,  2.18s/it]

Val:  10%|█         | 6/60 [00:20<01:23,  1.55s/it]

Val:  12%|█▏        | 7/60 [00:20<01:01,  1.16s/it]

Val:  13%|█▎        | 8/60 [00:20<00:46,  1.11it/s]

Val:  15%|█▌        | 9/60 [00:21<00:36,  1.38it/s]

Val:  17%|█▋        | 10/60 [00:21<00:30,  1.65it/s]

Val:  18%|█▊        | 11/60 [00:21<00:25,  1.91it/s]

Val:  20%|██        | 12/60 [00:22<00:22,  2.17it/s]

Val:  22%|██▏       | 13/60 [00:22<00:19,  2.43it/s]

Val:  23%|██▎       | 14/60 [00:22<00:17,  2.62it/s]

Val:  25%|██▌       | 15/60 [00:23<00:16,  2.74it/s]

Val:  27%|██▋       | 16/60 [00:23<00:15,  2.77it/s]

Val:  28%|██▊       | 17/60 [00:23<00:14,  2.88it/s]

Val:  30%|███       | 18/60 [00:27<00:56,  1.35s/it]

Val:  32%|███▏      | 19/60 [00:27<


        Validation of epoch: 6 is done; 
        loss: 0.4101; acc: 0.89, f1: 0.88040803
        



Train:  40%|███▉      | 135/339 [02:32<03:49,  1.13s/it]


KeyboardInterrupt: ignored